In [1]:
import pandas as pd

In [2]:
def import_excel(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)  
    #df = pd.concat(df, axis=0, ignore_index=True)
    return df

In [3]:
file_path = './data/plz/plz_per_lor.xls'
sheet_name = 'all'

plz = import_excel(file_path, sheet_name)

In [4]:
#dropping rows with no PLZ
plz['PLZ'].dropna(axis=0, inplace=True)

In [5]:
# group df by LOR and create column containing list of all PLZ of the LOR

plz_per_lor = plz.groupby(['LOR', 'PLZ'], as_index=False).count() \
                 .groupby('LOR')['PLZ'].apply(list).reset_index(name='all_PLZ')

plz_per_lor['LOR'] = plz_per_lor['LOR'].apply('{:0>6}'.format)

In [6]:
# find max number of PLZs in LORs to create correct amount of columns

plz_per_lor['count'] = plz_per_lor['all_PLZ'].apply(lambda x: len(x))
plz_per_lor['count'].max()

5

In [7]:
plz_per_lor[['plz1','plz2', 'plz3', 'plz4', 'plz5']] = pd.DataFrame(plz_per_lor['all_PLZ'].tolist(), index= plz_per_lor.index)

plz_per_lor.drop(['all_PLZ', 'count'], axis=1, inplace=True)

In [8]:
plz_per_lor

,LOR,plz1,plz2,plz3,plz4,plz5
0,010111,10785.0,10787.0,NaN,NaN,NaN
1,010112,10117.0,NaN,NaN,NaN,NaN
2,010113,10117.0,10178.0,10179.0,NaN,NaN
3,010114,10115.0,10119.0,NaN,NaN,NaN
4,010221,10551.0,10553.0,10555.0,NaN,NaN
...,...,...,...,...,...,...
133,122311,13465.0,13467.0,NaN,NaN,NaN
134,123012,13437.0,13469.0,NaN,NaN,NaN
135,123021,13435.0,13439.0,NaN,NaN,NaN
136,123022,13435.0,13469.0,NaN,NaN,NaN


In [9]:
import pgeocode

In [10]:
nomi = pgeocode.Nominatim('de')

In [11]:
plz_per_lor['lon'] = plz_per_lor['plz1'].apply(lambda x: nomi.query_postal_code(str(int(x)))['longitude'])
plz_per_lor['lat'] = plz_per_lor['plz1'].apply(lambda x: nomi.query_postal_code(str(int(x)))['latitude'])

In [12]:
plz_per_lor

,LOR,plz1,plz2,plz3,plz4,plz5,lon,lat
0,010111,10785.0,10787.0,NaN,NaN,NaN,13.3643,52.5073
1,010112,10117.0,NaN,NaN,NaN,NaN,13.3872,52.5170
2,010113,10117.0,10178.0,10179.0,NaN,NaN,13.3872,52.5170
3,010114,10115.0,10119.0,NaN,NaN,NaN,13.3846,52.5323
4,010221,10551.0,10553.0,10555.0,NaN,NaN,13.3372,52.5307
...,...,...,...,...,...,...,...,...
133,122311,13465.0,13467.0,NaN,NaN,NaN,13.2896,52.6399
134,123012,13437.0,13469.0,NaN,NaN,NaN,13.3284,52.5905
135,123021,13435.0,13439.0,NaN,NaN,NaN,13.3456,52.6020
136,123022,13435.0,13469.0,NaN,NaN,NaN,13.3456,52.6020


In [13]:
import joblib

joblib.dump(plz_per_lor, './dataframes/final_plz.pkl')

['./dataframes/final_plz.pkl']